In [4]:
import pandas as pd
import numpy as np

In [5]:
dataset_ftse_mib = pd.read_csv("./Datasets/FTSEMIB.MI.csv")

In [6]:
len(dataset_ftse_mib)

1268

In [7]:
def get_stocks_by_week(df, week):
    """
    Returns a dataframe of stocks for a given week
    """
    return df[df['Week Number'] == week]

In [8]:
def get_portfolio_weekly_return(portfolio):
  totalReturn = 0
  
  for index, row in portfolio.iterrows():
    log_ret = row["Stock Weekly Return"]
    totalReturn+=log_ret
  return totalReturn/len(portfolio)


In [9]:
def get_ftse_mib_weekly_return(week_number):
  ftse_mib = pd.read_csv("./Datasets/FTSEMIB.MI.csv")
  prev_week_ret = ftse_mib.iloc[180+week_number-7]["log_ret"]
  next_week_ret = ftse_mib.iloc[180+week_number]["log_ret"]
  return next_week_ret-prev_week_ret


In [10]:
test = pd.read_csv("./results.csv")

In [52]:
res = get_stocks_by_week(test, 0)
res = res[res["Stock Symbol"] != "FTSEMIB.MI"]
sorted_df = res.sort_values(by="Beta", ascending=False)
sorted_df.head(20)


,Unnamed: 0,Stock Symbol,Week Number,Alpha,Beta,R-Squared,Res Std. Error,Stock Weekly Return
5068,5068,STLA.MI,0,0.002795,1.822867,0.501008,0.041383,-0.008591
2184,2184,EXO.MI,0,0.000536,1.566642,0.624995,0.030567,0.005022
624,624,BAMI.MI,0,0.000089,1.360341,0.281186,0.023047,0.008328
5224,5224,STM.MI,0,0.000607,1.357758,0.283644,0.022959,-0.000041
5848,5848,UCG.MI,0,-0.000047,1.329340,0.448922,0.022008,-0.006662
1092,1092,BPE.MI,0,-0.000089,1.290830,0.241391,0.020751,0.020998
3432,3432,LDO.MI,0,-0.003591,1.193388,0.153714,0.017737,0.009069
2340,2340,FBK.MI,0,0.001319,1.186950,0.456557,0.017546,0.006053
1248,1248,BZU.MI,0,-0.000810,1.180497,0.378496,0.017356,-0.025273
1404,1404,CNHI.MI,0,0.000359,1.159120,0.413090,0.016733,-0.018219


Test: take top 15 and bottom 15 based on beta

In [53]:
top_10 = sorted_df.head(5)
bottom_10 = sorted_df.tail(5)

In [54]:
frames = [sorted_df.head(5),  sorted_df.tail(5)]
result = pd.concat(frames)


In [55]:
get_portfolio_weekly_return(result)


-0.0051364081407437815

In [56]:
get_ftse_mib_weekly_return(0)

-0.008566399982807801

In [57]:
for weekNumber in range(0,len(dataset_ftse_mib),7):
  print(weekNumber)

0
7
14
21
28
35
42
49
56
63
70
77
84
91
98
105
112
119
126
133
140
147
154
161
168
175
182
189
196
203
210
217
224
231
238
245
252
259
266
273
280
287
294
301
308
315
322
329
336
343
350
357
364
371
378
385
392
399
406
413
420
427
434
441
448
455
462
469
476
483
490
497
504
511
518
525
532
539
546
553
560
567
574
581
588
595
602
609
616
623
630
637
644
651
658
665
672
679
686
693
700
707
714
721
728
735
742
749
756
763
770
777
784
791
798
805
812
819
826
833
840
847
854
861
868
875
882
889
896
903
910
917
924
931
938
945
952
959
966
973
980
987
994
1001
1008
1015
1022
1029
1036
1043
1050
1057
1064
1071
1078
1085
1092
1099
1106
1113
1120
1127
1134
1141
1148
1155
1162
1169
1176
1183
1190
1197
1204
1211
1218
1225
1232
1239
1246
1253
1260
1267


In [58]:
final_dataframe = pd.DataFrame(columns=[
                               "Week", "stockSymbols", "portfolio Weekly Returns", "Ftse Mib weekly returns"])
results = pd.read_csv("./results.csv")
for weekNumber in range(0,len(dataset_ftse_mib),7):
  res = get_stocks_by_week(test, weekNumber)
  sorted_df = res.sort_values(by="Res Std. Error", ascending=True)
  frames = [sorted_df.head(4),  sorted_df.tail(4)]
  top_bottom_10 = pd.concat(frames)
  if len(top_bottom_10) > 0:
    portfolio_return = get_portfolio_weekly_return(top_bottom_10)
    ftse_mib_return = get_ftse_mib_weekly_return(weekNumber)
    final_dataframe.loc[len(final_dataframe)] = [
        weekNumber, top_bottom_10["Stock Symbol"].values, portfolio_return, ftse_mib_return]


In [59]:
final_dataframe.head()

,Week,stockSymbols,portfolio Weekly Returns,Ftse Mib weekly returns
0,0,"[PIRC.MI, NEXI.MI, PST.MI, CPR.MI, STM.MI, BAM...",-0.003105,-0.008566
1,7,"[G.MI, BZU.MI, SRG.MI, ENI.MI, PIRC.MI, NEXI.M...",-0.013727,-0.023240
2,14,"[REC.MI, FBK.MI, MONC.MI, IG.MI, NEXI.MI, DIA....",0.015360,0.020911
3,21,"[BZU.MI, EXO.MI, MONC.MI, IG.MI, PIRC.MI, DIA....",-0.008995,-0.015410
4,28,"[MONC.MI, UNI.MI, PRY.MI, SRG.MI, LDO.MI, AZM....",0.008663,0.015685


In [60]:
import plotly.express as px


In [61]:
fig = px.line(final_dataframe, x="Week", y=["portfolio Weekly Returns", "Ftse Mib weekly returns"],
              title='Returns')
fig.show()

In [62]:
import math


math.exp(-0.003105)


0.99689981552714

In [63]:
final_dataframe.head(1)

,Week,stockSymbols,portfolio Weekly Returns,Ftse Mib weekly returns
0,0,"[PIRC.MI, NEXI.MI, PST.MI, CPR.MI, STM.MI, BAM...",-0.003105,-0.008566


In [64]:
initial_investment = 100
portfolio_total_return = 100
index_total_return = 100
first = True
final_returns = pd.DataFrame(columns=["Week","Portfolio_Ret", "FTSE_MIB_Ret"])
for index, row in final_dataframe.iterrows():
  
  portfolio_total_return *= math.exp(row["portfolio Weekly Returns"])
  index_total_return *= math.exp(row["Ftse Mib weekly returns"])
 
  final_returns.loc[len(final_returns)] = [row["Week"],portfolio_total_return, index_total_return]
  

In [65]:
final_returns.head()

,Week,Portfolio_Ret,FTSE_MIB_Ret
0,0.0,99.689944,99.147019
1,7.0,98.330873,96.869363
2,14.0,99.852910,98.916344
3,21.0,98.958783,97.403750
4,28.0,99.819798,98.943551


In [72]:
print(final_returns["Portfolio_Ret"].var())
print(final_returns["FTSE_MIB_Ret"].var())


13.053659371217718
2.523307717776255


In [66]:
# first = True
# final_returns = pd.DataFrame(columns=["Week","Portfolio_Ret", "FTSE_MIB_Ret"])
# for index, row in final_dataframe.iterrows():
  
#   return_portfolio = initial_investment * \
#      ( math.exp(row["portfolio Weekly Returns"])/100) if first else portfolio_total_return * (math.exp(row["portfolio Weekly Returns"])/100)
#   return_index = initial_investment * (math.exp(row["Ftse Mib weekly returns"])/100)  if first else index_total_return *( math.exp(row["portfolio Weekly Returns"])/100)
#   if row["portfolio Weekly Returns"] < 0:
#     return_portfolio *= -1
#   if row["Ftse Mib weekly returns"] < 0:
#     return_index *= -1
#   if first:
#     portfolio_total_return += initial_investment+return_portfolio
#     index_total_return += initial_investment+return_index
#     first = False
#   else:
#     portfolio_total_return += return_portfolio
#     index_total_return += return_index
#   final_returns.loc[len(final_returns)] = [row["Week"],portfolio_total_return, index_total_return]


In [67]:
# first = True
# final_returns = pd.DataFrame(columns=["Week","Portfolio_Ret", "FTSE_MIB_Ret"])
# for index, row in final_dataframe.iterrows():
  
  
#   p_perc_return = (math.exp(row["portfolio Weekly Returns"]))
#   i_perc_return = (math.exp(row["Ftse Mib weekly returns"]))
#   if row["portfolio Weekly Returns"] < 0:
#     p_perc_return *=-1
#   if row["Ftse Mib weekly returns"]< 0:
#     i_perc_return *=-1
#   portfolio_total_return = initial_investment * \
#       p_perc_return if first else portfolio_total_return*p_perc_return
#   index_total_return = initial_investment * \
#       i_perc_return if first else index_total_return*i_perc_return
#   first = False
#   final_returns.loc[len(final_returns)] = [row["Week"],portfolio_total_return, index_total_return]

In [68]:
final_returns.head(36)

,Week,Portfolio_Ret,FTSE_MIB_Ret
0,0.0,99.689944,99.147019
1,7.0,98.330873,96.869363
2,14.0,99.852910,98.916344
3,21.0,98.958783,97.403750
4,28.0,99.819798,98.943551
5,35.0,99.388807,99.345388
6,42.0,99.860089,99.454101
7,49.0,100.749089,100.356524
8,56.0,101.302240,99.550320
9,63.0,99.761558,98.532928


In [69]:
fig = px.line(final_returns, x="Week", y=["Portfolio_Ret", "FTSE_MIB_Ret"],
              title='Returns')
fig.show()
